# Im2Spec Perovskite IV

This notebook demonstrates a new version of im2spec model where one trains jointly image-to-image and spec-to-spec variational autoencoders (VAEs) while trying to 'align' their respective latent spaces. The latent space 'alignment' is done with two methods: *i)* using cosine similarity as an extra loss term, and *ii)* learning the linear transformation map between the two latent spaces during the model training. Here we also add rotational and (optionally) translational invariance to image-to-image VAE.

We analyze hybrid perovskite IV dataset (Yongtao Liu and Jiawei Gong)

# Download:

In [ ]:
!pip install atomai
!pip install sidpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: Operation cancelled by user
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.3/516.3 kB 23.4 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)


# Import

In [ ]:
from typing import Union, Tuple, List, Type, Optional

import math
import matplotlib.pyplot as plt
import numpy as np
from atomai import utils

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

from scipy.signal import find_peaks
import h5py
import sidpy

In [ ]:
#@title convEncoderNet { form-width: "20%" }

from warnings import warn, filterwarnings
filterwarnings("ignore", module="torch.nn.functional")
tt = torch.tensor

########################### Convolutional neural nets #########################

class convEncoderNet(nn.Module):
    """
    Standard convolutional encoder
    """
    def __init__(self,
                 input_dim: Tuple[int],
                 input_channels: int = 1,
                 latent_dim: int = 2,
                 hidden_dim: List[Tuple[int]] = None,
                 batchnorm: bool = True,
                 activation: str = "lrelu",
                 pool: bool = True,
                 ) -> None:
        """
        Initializes encoder module
        """
        super(convEncoderNet, self).__init__()
        if hidden_dim is None:
            hidden_dim = [(32,), (64, 64), (96, 96)]
        output_dim = [dim // 2**len(hidden_dim) for dim in input_dim]
        output_channels = hidden_dim[-1][-1]
        self.latent_dim = latent_dim
        self.feature_extractor = FeatureExtractor(
            len(input_dim), input_channels, hidden_dim,
            batchnorm, activation, pool)
        self.features2latent = features_to_latent(
            [output_channels, *output_dim], 2*latent_dim)

    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor]:
        """
        Forward pass
        """
        x = self.feature_extractor(x)
        encoded = self.features2latent(x)
        mu, log_sig = encoded.split(self.latent_dim, 1)
        return mu, log_sig


class convDecoderNet(nn.Module):
    """
    Standard convolutional decoder
    """
    def __init__(self,
                 latent_dim: int,
                 output_dim: int,
                 output_channels: int = 1,
                 hidden_dim: List[Tuple[int]] = None,
                 batchnorm: bool = True,
                 activation: str = "lrelu",
                 sigmoid_out: bool = True,
                 upsampling_mode: str = "bilinear",
                 ) -> None:
        """
        Initializes decoder module
        """
        super(convDecoderNet, self).__init__()
        if hidden_dim is None:
            hidden_dim = [(96, 96), (64, 64), (32,)]
        input_dim = [dim // 2**len(hidden_dim) for dim in output_dim]
        self.latent2features = latent_to_features(
            latent_dim, [hidden_dim[0][0], *input_dim])
        self.upsampler = Upsampler(
            len(output_dim), hidden_dim[0][0], hidden_dim, output_channels,
            batchnorm, activation, upsampling_mode)
        self.activation_out = nn.Sigmoid() if sigmoid_out else lambda x: x

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass
        """
        x = self.latent2features(x)
        x = self.activation_out(self.upsampler(x))
        return x


class ConvBlock(nn.Module):
    """
    Creates a block of layers each consisting of convolution operation,
    (optional) nonlinear activation and (optional) batch normalization
    """
    def __init__(self,
                 ndim: int,
                 input_channels: int,
                 hidden_dim: Tuple[int],
                 kernel_size: Union[Tuple[int], int] = 3,
                 stride: Union[Tuple[int], int] = 1,
                 padding: Union[Tuple[int], int] = 1,
                 batchnorm: bool = False,
                 activation: str = "lrelu",
                 pool: bool = False,
                 ) -> None:
        """
        Initializes module parameters
        """
        super(ConvBlock, self).__init__()
        if not 0 < ndim < 4:
            raise AssertionError("ndim must be equal to 1, 2 or 3")
        activation = get_activation(activation)
        hidden_dim = (input_channels,) + hidden_dim
        block = []
        for in_channels, out_channels in zip(hidden_dim, hidden_dim[1:]):
            block.append(get_conv(ndim)(in_channels, out_channels,
                         kernel_size=kernel_size, stride=stride, padding=padding))
            if activation is not None:
                block.append(activation())
            if batchnorm:
                block.append(get_bnorm(ndim)(out_channels))
        if pool:
            block.append(get_maxpool(ndim)(2, 2))
        self.block = nn.Sequential(*block)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Defines a forward pass
        """
        output = self.block(x)
        return output


class UpsampleBlock(nn.Module):
    """
    Upsampling performed using bilinear or nearest-neigbor interpolation
    followed by 1-by-1 convolution, which an be used to reduce a number of
    feature channels
    """
    def __init__(self,
                 ndim: int,
                 input_channels: int,
                 output_channels: int,
                 scale_factor: int = 2,
                 mode: str = "bilinear") -> None:
        """
        Initializes module parameters
        """
        super(UpsampleBlock, self).__init__()
        warn_msg = ("'bilinear' mode is not supported for 1D and 3D;" +
                    " switching to 'nearest' mode")
        if mode not in ("bilinear", "nearest"):
            raise NotImplementedError(
                "Use 'bilinear' or 'nearest' for upsampling mode")
        if not 0 < ndim < 4:
            raise AssertionError("ndim must be equal to 1, 2 or 3")
        if mode == "bilinear" and ndim in (3, 1):
            warn(warn_msg, category=UserWarning)
            mode = "nearest"
        self.mode = mode
        self.scale_factor = scale_factor
        self.conv = get_conv(ndim)(
            input_channels, output_channels,
            kernel_size=1, stride=1, padding=0)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Defines a forward pass
        """
        x = F.interpolate(
            x, scale_factor=self.scale_factor, mode=self.mode)
        return self.conv(x)


class FeatureExtractor(nn.Sequential):
    """
    Convolutional feature extractor
    """
    def __init__(self,
                 ndim: int,
                 input_channels: int = 1,
                 hidden_dim: List[Tuple[int]] = None,
                 batchnorm: bool = True,
                 activation: str = "lrelu",
                 pool: bool = True,
                 ) -> None:
        """
        Initializes feature extractor module
        """
        super(FeatureExtractor, self).__init__()
        if hidden_dim is None:
            hidden_dim = [[16,], (32, 32), (64, 64)]
        for i, hdim in enumerate(hidden_dim):
            in_filters = input_channels if i == 0 else hidden_dim[i-1][-1]
            block = ConvBlock(ndim, in_filters, hdim,
                              batchnorm=batchnorm, activation=activation,
                              pool=pool)
            self.add_module("c{}".format(i), block)


class Upsampler(nn.Sequential):
    """
    Convolutional upsampler
    """
    def __init__(self,
                 ndim: int,
                 input_channels: int,
                 hidden_dim: List[Tuple[int]] = None,
                 output_channels: int = 1,
                 batchnorm: bool = True,
                 activation: str = "lrelu",
                 upsampling_mode: str = "bilinear",
                 ) -> None:
        """
        Initializes upsampler module
        """
        super(Upsampler, self).__init__()
        if hidden_dim is None:
            hidden_dim = [(64, 64), (32, 32), (16,)]
        upsampling_mode = 'nearest' if ndim in (1, 3) else upsampling_mode
        for i, hdim in enumerate(hidden_dim):
            in_filters = input_channels if i == 0 else hidden_dim[i-1][-1]
            block = ConvBlock(ndim, in_filters, hdim,
                              batchnorm=batchnorm, activation=activation,
                              pool=False)
            self.add_module("conv_block_{}".format(i), block)
            up = UpsampleBlock(ndim, hdim[-1], hdim[-1], mode=upsampling_mode)
            self.add_module("up_{}".format(i), up)

        out = ConvBlock(ndim, hdim[-1], (1,), 1, 1, 0, activation=None)
        self.add_module("output_layer", out)


class features_to_latent(nn.Module):
    """
    Maps features (usually, from a convolutional net/layer) to latent space
    """
    def __init__(self, input_dim: Tuple[int], latent_dim: int = 2) -> None:
        super(features_to_latent, self).__init__()
        self.reshape_ = torch.prod(tt(input_dim))
        self.fc_latent = nn.Linear(self.reshape_, latent_dim)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x.view(-1, self.reshape_)
        return self.fc_latent(x)


class latent_to_features(nn.Module):
    """
    Maps latent vector to feature space
    """
    def __init__(self, latent_dim: int, out_dim: Tuple[int]) -> None:
        super(latent_to_features, self).__init__()
        self.reshape_ = out_dim
        self.fc = nn.Linear(latent_dim, torch.prod(tt(out_dim)).item())

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.fc(x)
        return x.view(-1, *self.reshape_)

def get_bnorm(dim: int) -> Type[nn.Module]:
    bn_dict = {1: nn.BatchNorm1d, 2: nn.BatchNorm2d, 3: nn.BatchNorm3d}
    return bn_dict[dim]


def get_conv(dim: int) -> Type[nn.Module]:
    conv_dict = {1: nn.Conv1d, 2: nn.Conv2d, 3: nn.Conv3d}
    return conv_dict[dim]


def get_maxpool(dim: int) -> Type[nn.Module]:
    conv_dict = {1: nn.MaxPool1d, 2: nn.MaxPool2d, 3: nn.MaxPool3d}
    return conv_dict[dim]



########################## Fully-connected neural nets #########################

class fcEncoderNet(nn.Module):
    """
    Standard fully-connected encoder NN for VAE.
    The encoder outputs mean and standard evidation of the encoded distribution.
    """
    def __init__(self,
                 in_dim: Tuple[int],
                 latent_dim: int = 2,
                 input_channels: int = 1,
                 hidden_dim: List[int] = None,
                 activation: str = 'tanh',
                 softplus_out: bool = False,
                 flat: bool = True
                 ) -> None:
        """
        Initializes module
        """
        super(fcEncoderNet, self).__init__()
        if len(in_dim) not in [1, 2]:
            raise ValueError("in_dim must be (h, w) or (l,)")
        if hidden_dim is None:
            hidden_dim = [256, 256]
        self.in_dim = torch.prod(tt(in_dim)).item() * input_channels

        self.fc_layers = make_fc_layers(
            self.in_dim, hidden_dim, activation)
        self.fc11 = nn.Linear(hidden_dim[-1], latent_dim)
        self.fc12 = nn.Linear(hidden_dim[-1], latent_dim)
        self.activation_out = nn.Softplus() if softplus_out else lambda x: x

    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor]:
        """
        Forward pass
        """
        x = x.view(-1, self.in_dim)
        x = self.fc_layers(x)
        mu = self.fc11(x)
        sigma = self.activation_out(self.fc12(x))
        return mu, sigma


class fcDecoderNet(nn.Module):
    """
    Standard fully-connected decoder for VAE
    """
    def __init__(self,
                 latent_dim: int,
                 output_dim: Tuple[int],
                 output_channels: int = 1,
                 hidden_dim: List[int] = None,
                 activation: str = 'tanh',
                 sigmoid_out: bool = True,
                 ) -> None:
        """
        Initializes module
        """
        super(fcDecoderNet, self).__init__()
        if len(output_dim) not in [1, 2]:
            raise ValueError("in_dim must be (h, w) or (l,)")
        if hidden_dim is None:
            hidden_dim = [256, 256]
        self.reshape = (output_channels,) + (output_dim)
        hidden_out = torch.prod(tt(output_dim)).item() * output_channels

        self.fc_layers = make_fc_layers(
            latent_dim, hidden_dim, activation)
        self.out = nn.Linear(hidden_dim[-1], hidden_out)
        self.activation_out = nn.Sigmoid() if sigmoid_out else lambda x: x

    def forward(self, z: torch.Tensor) -> torch.Tensor:
        """
        Forward pass
        """
        x = self.fc_layers(z)
        x = self.activation_out(self.out(x))
        return x.view(-1, *self.reshape)


class sDecoderNet(nn.Module):
    """
    Spatial generator (decoder) network with fully-connected layers
    """
    def __init__(self,
                 latent_dim: int,
                 output_dim: Tuple[int],
                 output_channels: int = 1,
                 hidden_dim: List[int] = None,
                 activation: str = 'tanh',
                 sigmoid_out: bool = True,
                 ) -> None:
        """
        Initializes module
        """
        super(sDecoderNet, self).__init__()
        if len(output_dim) not in [1, 2]:
            raise ValueError("in_dim must be (h, w) or (l,)")
        if hidden_dim is None:
            hidden_dim = [256, 256]
        self.reshape = (output_channels,) + (output_dim)
        coord_dim = 1 if len(output_dim) < 2 else 2

        self.coord_latent = coord_latent(
            latent_dim, hidden_dim[0], coord_dim)
        self.fc_layers = make_fc_layers(
            hidden_dim[0], hidden_dim, activation)
        self.out = nn.Linear(hidden_dim[-1], output_channels)
        self.activation_out = nn.Sigmoid() if sigmoid_out else lambda x: x

    def forward(self, x_coord: torch.Tensor, z: torch.Tensor) -> torch.Tensor:
        """
        Forward pass
        """
        x = self.coord_latent(x_coord, z)
        x = self.fc_layers(x)
        x = self.activation_out(self.out(x))
        return x.view(-1, *self.reshape)


class coord_latent(nn.Module):
    """
    The "spatial" part of the iVAE's decoder that allows for translational
    and rotational invariance (based on https://arxiv.org/abs/1909.11663)
    """
    def __init__(self,
                 latent_dim: int,
                 out_dim: int,
                 ndim: int = 2,
                 activation_out: bool = True) -> None:
        """
        Initializes module
        """
        super(coord_latent, self).__init__()
        self.fc_coord = nn.Linear(ndim, out_dim)
        self.fc_latent = nn.Linear(latent_dim, out_dim, bias=False)
        self.activation = nn.Tanh() if activation_out else None

    def forward(self,
                x_coord: torch.Tensor,
                z: Tuple[torch.Tensor]) -> torch.Tensor:
        batch_dim, n = x_coord.size()[:2]
        x_coord = x_coord.reshape(batch_dim * n, -1)
        h_x = self.fc_coord(x_coord)
        h_x = h_x.reshape(batch_dim, n, -1)
        h_z = self.fc_latent(z)

        h_z = h_z.view(-1, h_z.size(-1))
        h = h_x.add(h_z.unsqueeze(1))
        h = h.reshape(batch_dim * n, -1)
        if self.activation is not None:
            h = self.activation(h)
        return h


def make_fc_layers(in_dim: int,
                   hidden_dim: List[int] = 128,
                   activation: str = "tanh"
                   ) -> Type[nn.Module]:
    """
    Generates a module with stacked fully-connected (aka dense) layers
    """
    fc_layers = []
    hidden_dim = [in_dim] + hidden_dim
    for h_in, h_out in zip(hidden_dim, hidden_dim[1:]):
        fc_layers.extend(
            [nn.Linear(h_in, h_out),
             get_activation(activation)()]),
             #get_bnorm(1)(h_out)])
    fc_layers = nn.Sequential(*fc_layers)
    return fc_layers


########################### Other utility functions ##########################

def grid2xy(X1: torch.Tensor, X2: torch.Tensor) -> torch.Tensor:
    X = torch.cat((X1[None], X2[None]), 0)
    d0, d1 = X.shape[0], X.shape[1] * X.shape[2]
    X = X.reshape(d0, d1).T
    return X


def imcoordgrid(im_dim: Tuple[int]) -> torch.Tensor:
    xx = torch.linspace(-1, 1, im_dim[0])
    yy = torch.linspace(1, -1, im_dim[1])
    x0, x1 = torch.meshgrid(xx, yy)
    return grid2xy(x0, x1)


def generate_grid(data_dim: Tuple[int]) -> torch.Tensor:
    """Generates 1D or 2D grid of coordinates. Returns a torch tensor with two
    axes. If the input data_dim indicates only one dimensional data, then the
    output will be a 2d torch tensor artificially augmented along the last
    dimension, of shape [N, 1].
    Args:
        data_dim:
            Dimensions of the input data.
    Raises:
        NotImplementedError:
            If the dimension (length) of the provided data_dim is not equal to
            1 or 2.
    Returns:
        The grid (always 2d).
    """

    if len(data_dim) not in [1, 2]:
        raise NotImplementedError("Currently supports only 1D and 2D data")
    if len(data_dim) == 1:
        return torch.linspace(-1, 1, data_dim[0])[:, None]
    return imcoordgrid(data_dim)


def transform_coordinates(coord: torch.Tensor,
                          phi: Union[torch.Tensor, float] = 0,
                          coord_dx: Union[torch.Tensor, float] = 0,
                          scale: Union[torch.Tensor, float] = 1.,
                          ) -> torch.Tensor:
    """
    Rotation of 2D coordinates followed by scaling and translation.
    For 1D grid, there is only transaltion. Operates on batches.
    """
    if coord.shape[-1] == 1:
        return coord + coord_dx
    coord = rotate_coordinates(coord, phi)
    return coord + coord_dx


def rotate_coordinates(coord: torch.Tensor,
                       phi: torch.Tensor,
                       ) -> torch.Tensor:
    """
    Rotation of 2D coordinates. Operates on batches
    """
    rotmat_r1 = torch.stack([torch.cos(phi), torch.sin(phi)], 1)
    rotmat_r2 = torch.stack([-torch.sin(phi), torch.cos(phi)], 1)
    rotmat = torch.stack([rotmat_r1, rotmat_r2], axis=1)
    coord = torch.bmm(coord, rotmat)
    return coord

def get_activation(activation: int) -> Type[nn.Module]:
    if activation is None:
        return
    activations = {"lrelu": nn.LeakyReLU, "tanh": nn.Tanh,
                   "softplus": nn.Softplus, "relu": nn.ReLU,
                   "gelu": nn.GELU}
    return activations[activation]


def init_dataloader(*args: torch.Tensor,
                    random_sampler: bool = False,
                    shuffle: bool = True,
                    **kwargs: int
                    ) -> Type[torch.utils.data.DataLoader]:
    """
    Returns initialized PyTorch dataloader, which is used by pyroVED's trainers.
    The inputs are torch Tensor objects containing training data and (optionally)
    labels.

    Example:
    >>> # Load training data stored as numpy array
    >>> train_data = np.load("my_training_data.npy")
    >>> # Transform numpy array to toech Tensor object
    >>> train_data = torch.from_numpy(train_data).float()
    >>> # Initialize dataloader
    >>> train_loader = init_dataloader(train_data)
    """
    batch_size = kwargs.get("batch_size", 100)
    tensor_set = torch.utils.data.dataset.TensorDataset(*args)
    if random_sampler:
        sampler = torch.utils.data.RandomSampler(tensor_set)
        data_loader = torch.utils.data.DataLoader(
            dataset=tensor_set, batch_size=batch_size, sampler=sampler)
    else:
        data_loader = torch.utils.data.DataLoader(
            dataset=tensor_set, batch_size=batch_size, shuffle=shuffle)
    return data_loader


def generate_latent_grid(d: int, **kwargs) -> torch.Tensor:
    """
    Generates a grid of latent space coordinates
    """
    dist = torch.distributions
    if isinstance(d, int):
        d = [d, d]
    z_coord = kwargs.get("z_coord")
    if z_coord:
        z1, z2, z3, z4 = z_coord
        grid_x = torch.linspace(z2, z1, d[0])
        grid_y = torch.linspace(z3, z4, d[1])
    else:
        grid_x = dist.Normal(0, 1).icdf(torch.linspace(0.95, 0.05, d[0]))
        grid_y = dist.Normal(0, 1).icdf(torch.linspace(0.05, 0.95, d[1]))
    z = []
    for xi in grid_x:
        for yi in grid_y:
            z.append(tt([xi, yi]).float().unsqueeze(0))
    return torch.cat(z), (grid_x, grid_y)


def set_torch_seed(seed: int = 1) -> None:
    """Sets all torch manual seeds"""
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

# Download data

In [ ]:
!pip install --upgrade gdown
!gdown -q https://drive.google.com/uc?id=1_llkLoA2SgcrT1NTvL04M5wHO7XxOpB6

In [ ]:
# unzip
!unzip -j /content/IV_data.zip;

### Load topography

In [ ]:
topoh5 = h5py.File("topo_6um_0.hf5")
sidpy.hdf_utils.print_tree(topoh5)

In [ ]:
topo = topoh5['BE Channels/Channels/Channels'][1, :,:,0]*260
mask1 = np.zeros((384, 384))
for i in range (384):
    mask1 [:,i]=topo.mean(1)
topo = topo - mask1
topo = topo - topo.min()

plt.figure(dpi = 300)
plt.imshow(topo, cmap = "gray")
plt.colorbar(label = "Height (nm)", shrink = 0.8)
plt.xticks([])
plt.yticks([])

### Load IV coordinates

In [ ]:
iv_coordinates = np.load("coordinates.npy")
print(iv_coordinates.shape)

plt.figure(dpi = 300)
plt.imshow(topo, cmap = 'gray')
plt.colorbar(label = "Height (nm)", shrink = 0.8)
plt.scatter(iv_coordinates[:,1], iv_coordinates[:,0], c ='r', s = 10)
plt.xticks([])
plt.yticks([])


### Plot one IV

In [ ]:
ivh5 = h5py.File("IV_light_n2V_100_0.hf5")
sidpy.hdf_utils.print_tree(ivh5)

In [ ]:
iv = ivh5['BE Channels/Channels/Channels'][0,0,]*100
iv = iv-iv.max()
vdc = ivh5['BEPS/vdc_waveform']
vdc = np.asarray(vdc)
plt.scatter(vdc, iv, c = np.arange(len(vdc)), cmap = 'jet', s = 10)


### Define a function to smooth IV

In [ ]:
# smooth/denoise iv
from scipy.signal import savgol_filter

def smooth(y):
  y_smooth = savgol_filter(y, 11, 3)
  return y_smooth

iv_smooth = smooth(iv)

plt.plot(vdc, iv_smooth)
plt.plot(vdc, iv)


### Load all IV and smooth them

In [ ]:
# load all iv data
ivdata = np.zeros((543, 128))  # 543 IV, length = 128
for i in range (543):
  ivh5 = h5py.File("IV_light_n2V_{}_0.hf5".format(i))
  iv = ivh5['BE Channels/Channels/Channels'][0,0,]*100
  # iv = iv-iv.max()
  ivdata[i,:] = iv

ivdata = ivdata - ivdata.min()

# smooth iv data
ivdata_smooth = np.zeros((543, 128))
for i in range (543):
  ivdata_smooth[i,:] = smooth(ivdata[i,:])

In [ ]:
vdc = ivh5['BEPS/vdc_waveform']
vdc = np.asarray(vdc)
plt.figure(figsize = (3, 2.5), dpi = 300)

plt.plot(vdc, ivdata_smooth[100,:]-ivdata_smooth[100,:].max(), c = "r")
plt.plot(vdc, ivdata_smooth[110,:]-ivdata_smooth[110,:].max(), c = "b")
plt.plot(vdc, ivdata_smooth[120,:]-ivdata_smooth[120,:].max(), c = "g")

plt.yticks([0.0, -0.1, -0.2, -0.3])
plt.xticks([0.0, -1.0, -2.0])

plt.xlabel ("DC Voltage (V)")
plt.ylabel ("Current (nA)")

# Im2Spec Analysis

### Extract image patches

In [ ]:
# Normalize all structure
norm_ = lambda x: (x - x.min()) / (x.max() - x.min())

image = topo

windowsize = 28  # patch size
img, com, _ = utils.extract_subimages(image, iv_coordinates, windowsize)
img = img[:,:,:,0]

indices = np.asarray(com)

img.shape

In [ ]:
plotid = 100
f, (ax1, ax2) = plt.subplots(1,2, figsize = (15, 5))
ax1.imshow(image, cmap = 'gray')
ax1.scatter(com[plotid,1], com[plotid,0], s = 10)

ax2.imshow(img[plotid,], cmap = 'gray')

In [ ]:
# Find the index of used coordinate, so that we can select the corresponding IV
def find_same_item(arr1, arr2):
  for item in arr1:
    if item in arr2:
      return item

def find_sub_coor_index(array, sub_array):
  idx = []
  for i in range(sub_array.shape[0]):
    idx_x = np.where(array[:,1] == sub_array[i,1])
    idx_y = np.where(array[:,0] == sub_array[i,0])
    idx.append(find_same_item(idx_x[0], idx_y[0]))

  return np.asarray(idx, dtype = int)

# find the sub index
sub_idx = find_sub_coor_index(iv_coordinates, com)

# extract the corresponding iv
iv_train = ivdata_smooth[sub_idx,]
iv_train.shape

Show corresponding image patches and spectra

In [ ]:
k1 = 15; k2 = 300; k3 = 220; k4 = -15
plt.figure(dpi = 300)
plt.imshow(topo, cmap = 'gray')
plt.colorbar(label = "Height (nm)", shrink = 0.8)
plt.scatter(iv_coordinates[k1,1], iv_coordinates[k1,0], c ='r', s = 30)
plt.scatter(iv_coordinates[k2,1], iv_coordinates[k2,0], c ='b', s = 30)
plt.scatter(iv_coordinates[k3,1], iv_coordinates[k3,0], c ='g', s = 30)
plt.scatter(iv_coordinates[k4,1], iv_coordinates[k4,0], c ='orange', s = 30)
plt.xticks([])
plt.yticks([])

plt.figure(figsize = (4, 3), dpi = 300)
plt.plot(vdc, ivdata_smooth[k1,:]-ivdata_smooth[k1,:].max(), c = "r")
plt.plot(vdc, ivdata_smooth[k2,:]-ivdata_smooth[k2,:].max(), c = "b")
plt.plot(vdc, ivdata_smooth[k3,:]-ivdata_smooth[k3,:].max(), c = "g")
plt.plot(vdc, ivdata_smooth[k4,:]-ivdata_smooth[k4,:].max(), c = "orange")

plt.yticks([0.0, -0.2, -0.4, -0.6])
plt.xticks([0.0, -1.0, -2.0])

plt.ylim(-0.65, 0.05)

plt.xlabel ("DC Voltage (V)")
plt.ylabel ("Current (nA)")

In [ ]:
hys = []; max_current = [];
for i in range (len(ivdata_smooth)):
  half_spec_length = int(ivdata_smooth[i,].shape[0]/2)
  hys_i = np.abs(ivdata_smooth[i,:half_spec_length].sum()-ivdata_smooth[i,half_spec_length:].sum())
  hys.append(hys_i)
  max_current.append(ivdata_smooth[i,].max())
hys = norm_(np.asarray(hys))
max_current = norm_(np.asarray(max_current))

In [ ]:
plt.figure(dpi = 300)
plt.imshow(topo, cmap = 'gray')
plt.scatter(iv_coordinates[:,1], iv_coordinates[:,0], c =hys, cmap = "jet", s = 10, vmax = 0.2)
plt.xticks([])
plt.yticks([])
plt.colorbar(label = "Normalized Hysteresis (a.u.)", shrink = 0.8)

In [ ]:
plt.figure(dpi = 300)
plt.imshow(topo, cmap = 'gray')
plt.scatter(iv_coordinates[:,1], iv_coordinates[:,0], c =max_current, cmap = "jet", s = 10)
plt.xticks([])
plt.yticks([])
plt.colorbar(label = "Normalized Maximum Current (a.u.)", shrink = 0.8)

In [ ]:
k = [200, 300, 400]
for i in k:
  plt.figure(figsize = (4, 3), dpi = 300)
  plt.plot(vdc, ivdata_smooth[i,:]-ivdata_smooth[i,:].max(), c = "b")
  plt.xlabel ("DC Voltage (V)")
  plt.ylabel ("Current (nA)")
  plt.ylim(-0.55, 0.05)
  plt.yticks([0.0, -0.25, -0.5])
  plt.xticks([0.0, -1.0, -2.0])
  plt.show()

for i in k:
  plt.figure( dpi = 300)
  plt.imshow(img[i,], cmap = 'gray')
  plt.yticks([])
  plt.xticks([])
  plt.show()

In [ ]:
k = 111
_, (ax0, ax1, ax2) = plt.subplots(1,3, figsize = (15, 5))
ax0.imshow(image, cmap = 'gray')
ax0.scatter(com[k,1], com[k,0], s = 20, c= 'r')
ax1.imshow(img[k,], cmap = 'gray')
ax2.plot(vdc, iv_train[k,])

In [ ]:
# Normalize all structure
norm_ = lambda x: (x - x.min()) / (x.max() - x.min())

for i in range (img.shape[0]):
  img[i,] = norm_(img[i,])

## Prepare torch data for training

In [ ]:
imgdata = img
indices = com
specdata = iv_train
full_img = image

imgdata = torch.from_numpy(imgdata).float()
specdata = torch.from_numpy(specdata).float()

#specdata = F.avg_pool1d(specdata[:, None], 8, 8).squeeze()  # reduce the spectral size via average pooling

print(imgdata.shape, specdata.shape)

Now let's define the ImSpec base class. It consists of a 'constructor' ```__init__``` where we define our model parameters, methods for obtaining pdf-s (e.g., ```_log_normal```), computing different loss metrics (e.g. ```_compute_kld```), performing a reparametrization trick (```_reparameterize```), etc. These methods are going to be reused in all subsequent ImSpec models. Finally, there is a ```forward``` method, which defines a forward propagation of input data through our model. Here, the images will be passed through one VAE and the spectra are going to be passed through another VAE. The first VAE also enforces rotational and (optionally) translational invariance of the latent representation. At the moment, the two paths are not interacting. Note that our ```forward``` method also records all the computed losses.

In [ ]:
#@title imspec { form-width: "20%" }
class imspec(nn.Module):

    def __init__(self,
                 indim1: Tuple[int, int], # assumed to be images
                 indim2: Tuple[int], # assumed to be spectra
                 latent_dim: int = 2,  # for now, both VAEs will have the same number of latent dims
                 coord: int = 0, # 1 for rotation, 2 for translations, 3 for rotation+translations
                 **kwargs: torch.Tensor) -> None:
        """Initialize model parameters"""
        super(imspec, self).__init__()
        device = "cuda" if torch.cuda.is_available() else 'cpu'
        # encoder and decoder for first dataset (e.g. structural images)
        self.encoder1 = fcEncoderNet(indim1, latent_dim + coord)
        dnet = sDecoderNet if coord > 0 else fcDecoderNet
        self.decoder1 = dnet(latent_dim, indim1)
        # encoder and decoder for second dataset (e.g. spectra)
        self.encoder2 = fcEncoderNet(indim2, latent_dim)
        self.decoder2 = fcDecoderNet(latent_dim, indim2)
        # Initialize grid and get 'prior' parameters for coordinate transformations
        self.grid = generate_grid(indim1).to(device)
        self.coord = coord
        self.phi_prior = kwargs.get("phi_prior", tt(math.pi / 2)).to(device)
        self.dt_prior = kwargs.get("dt_prior", tt(0.5)).to(device)
        # Place all model parameters on the appropriate device
        self.to(device)
        self.device = device

    def _compute_kld(self,
                     z: torch.Tensor,
                     q_param: Tuple[torch.Tensor],
                     p_param: Optional[Tuple[torch.Tensor]] = None
                     ) -> torch.Tensor:
        """
        Computes KL divergence term between two normal distributions
        or (if p_param = None) between normal and standard normal distributions
        """
        qz = self._log_normal(z, *q_param)
        if p_param is None:
            pz = self._log_unit_normal(z)
        else:
            pz = self._log_normal(z, *p_param)
        return qz - pz

    @classmethod
    def _reparameterize(cls,
                        z_mu: torch.Tensor,
                        z_logsig: torch.Tensor
                        ) -> torch.Tensor:
        """Reparametrization trick"""
        batch_dim = z_mu.size(0)
        z_dim = z_mu.size(1)
        z_sig = torch.exp(z_logsig)
        eps = z_mu.new(batch_dim, z_dim).normal_()
        return z_mu + z_sig * eps

    @classmethod
    def _log_normal(cls,
                   x: torch.Tensor,
                   mu: torch.Tensor,
                   log_sig: torch.Tensor
                   ) -> torch.Tensor:
        """
        Computes log-pdf of a normal distribution
        """
        log_pdf = (-0.5 * math.log(2 * math.pi) - log_sig -
                   (x - mu)**2 / (2 * torch.exp(log_sig)**2))
        return torch.sum(log_pdf, dim=-1)

    @classmethod
    def _log_unit_normal(cls, x: torch.Tensor) -> torch.Tensor:
        """
        Computes log-pdf of a unit normal distribution
        """
        log_pdf = -0.5 * (math.log(2 * math.pi) + x ** 2)
        return torch.sum(log_pdf, dim=-1)

    @classmethod
    def _compute_re(cls,
                    x: torch.Tensor,
                    x_reconstr: torch.Tensor) -> torch.Tensor:
        """Computes binary cross-entropy reconstruction loss"""
        reconstr_loss = F.binary_cross_entropy(
            x_reconstr.flatten(1), x.flatten(1), reduction='none').sum(-1)
        return reconstr_loss

    def _transform_coordinates(self,
                               grid: torch.Tensor,
                               t_vec: torch.Tensor) -> torch.Tensor:
        """Rotation and translation of the 2d coordinate grid"""
        phi = t_vec[:, 0]
        dt = t_vec[:, 1:].unsqueeze(1) if self.coord > 1 else 0
        grid = rotate_coordinates(grid, phi)
        return grid + self.dt_prior * dt

    def forward(self, x1: torch.Tensor, x2: torch.Tensor=None) -> torch.Tensor:
        """Forward Pass"""

        self.likelihood = 0
        self.kl_divergence = 0

        # First VAE
        # Encode data
        z_mu1, z_logsig1 = self.encoder1(x1)
        # Obtain latent vector throught he 'reparametrization trick'
        z_vec1 = self._reparameterize(z_mu1, z_logsig1)
        # Split latent vector into parts associated with coordinate transformations and data content
        t_vec1 = z_vec1[:, :self.coord]
        z_vec1 = z_vec1[:, self.coord:]
        # Compute KLD for the 'conventional' latent vector
        self.kl_divergence += self._compute_kld(
            z_vec1, (z_mu1[:, self.coord:], z_logsig1[:, self.coord:]))
        # Compute KLD for the latent angle (if any)
        if self.coord in [1, 3]:
            p_loc = t_vec1.new_zeros(t_vec1.size(0), 1)
            p_scale = t_vec1.new_ones(t_vec1.size(0), 1) * self.phi_prior
            self.kl_divergence += self._compute_kld(
                t_vec1[:, :1], (z_mu1[:, :1], z_logsig1[:, :1]), (p_loc, p_scale))
            z_mu1, z_logsig1 = z_mu1[:, 1:], z_logsig1[:, 1:]
            t_vec1 = t_vec1[:, 1:] if t_vec1.size(-1) > 1 else t_vec1
        # Compute KLD for the latent shift (if any)
        if self.coord in [2, 3]:
            self.kl_divergence += self._compute_kld(
                t_vec1, (z_mu1[:, :2], z_logsig1[:, :2])) # assume unit normal prior
        # Do the affine transformation for the coordinate grid (if any)
        decoder_input = (z_vec1,)
        if self.coord:
            grid = self.grid.expand(t_vec1.shape[0], *self.grid.shape)
            grid = self._transform_coordinates(grid, t_vec1)
            decoder_input = (grid,) + decoder_input
        # Pass latent data and transformed grid (if any) to decoder
        x_reconstr1 = self.decoder1(*decoder_input)
        # Compute reconstruction loss
        self.likelihood += -self._compute_re(x1, x_reconstr1)

        # Second VAE (just vanilla VAE)
        z_mu2, z_logsig2 = self.encoder2(x2)
        z_vec2 = self._reparameterize(z_mu2, z_logsig2)
        self.kl_divergence += self._compute_kld(z_vec2, (z_mu2, z_logsig2))
        x_reconstr2 = self.decoder2(z_vec2)
        self.likelihood += -self._compute_re(x2, x_reconstr2)

        return (z_vec1, x_reconstr1), (z_vec2, x_reconstr2)

## Train two VAEs without enforcing similarity b/w latent spaces
Now let's proceed to model training. First, we simply train two VAEs (one for images, and another for spectra) without enforcing similarity between the latent spaces.

Initialize dataloader:

In [ ]:
imgdata = norm_(imgdata)
specdata = norm_(specdata)

train_loader = init_dataloader(imgdata.unsqueeze(1), specdata.unsqueeze(1), batch_size=200)  # unsqueeze(1) adds a channel dimension of 1

Initialize our model and an optimizer:

In [ ]:
img_dim = (imgdata.shape[-1], imgdata.shape[-1])
spec_dim = (specdata.shape[-1],)

set_torch_seed(1)
model1 = imspec(img_dim, spec_dim, latent_dim=2, coord=3)
optimizer = torch.optim.Adam(model1.parameters(), lr=1e-4)

Finaly, we train the model. It is usually convenient to define a function that performs a single training step and then call it from within the training loop.

In [ ]:
epochs = 40000

In [ ]:
def train_step1(x1: torch.Tensor, x2: torch.Tensor,
               beta: float = 1.) -> float:
    """ Single train step"""
    model1.train()
    _ = model1(x1.to(model1.device), x2.to(model1.device))
    loss = -(model1.likelihood - beta * model1.kl_divergence).mean()  # compute losses
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return loss.item()

# training loop
train_loss = []
for e in range(epochs):
    train_loss_i = 0
    for (x1, x2) in train_loader:
        train_loss_i += train_step1(x1, x2)
    train_loss.append(train_loss_i / len(train_loader))
    print("Epoch {},  Train loss {}".format(e+1, train_loss[-1]))

Encode data into the latent space(s) using the trained model:

In [ ]:
# initialize test dataloader (here it using the same data as the training dataloader but without random shuffling)
test_loader = init_dataloader(imgdata.unsqueeze(1), specdata.unsqueeze(1), shuffle=False)

model1.eval()  # switch to evaluation model (turn off batchnorm, etc.)
with torch.no_grad():  # we do not need to track gradients at the prediction stage
    z_mu1 = torch.cat([model1.encoder1(x.to(model1.device))[0].cpu() for (x,_) in test_loader])  # encoded image data
    z_mu2 = torch.cat([model1.encoder2(x.to(model1.device))[0].cpu() for (_,x) in test_loader])  # encoded spectral data

plt.scatter(z_mu1[:, -1], z_mu1[:, -2], s=24, alpha=0.3, label="image data")
plt.scatter(z_mu2[:, -1], z_mu2[:, -2], s=24, alpha=0.3, label="spectral data")
plt.legend()

Correlation analysis with two latents

In [ ]:
plt.scatter(z_mu1[:, -1], z_mu2[:, -1], s=24, alpha=0.3, label="2 vs 2")
plt.legend()

In [ ]:
plt.scatter(z_mu1[:, -2], z_mu2[:, -1], s=24, alpha=0.3, label="1 vs 2")
plt.legend()

In [ ]:
plt.scatter(z_mu1[:, -1], z_mu2[:, -2], s=24, alpha=0.3, label="2 vs 1")
plt.legend()

In [ ]:
plt.scatter(z_mu1[:, -2], z_mu2[:, -2], s=24, alpha=0.3, label="1 vs 1")
plt.legend()

Now let's view the learned latent manifold(s) by projecting them to the data space(s):

In [ ]:
d = 8  # grid size
z, (gridx, gridy) = generate_latent_grid(d)
coord_grid = model1.grid.expand(len(z), *model1.grid.shape)


with torch.no_grad():
    manifold1 = model1.decoder1(coord_grid, z.to(model1.device)).cpu()  # get the image data manifold
    manifold2 = model1.decoder2(z.to(model1.device)).cpu()  # get the spectral data manifold

In [ ]:
grid = torchvision.utils.make_grid(manifold1, nrow=d, padding=1, pad_value=0.8)[0]
plt.figure(figsize=(6,6), dpi=100)
plt.imshow(grid, cmap='gray')
plt.axis("off")

In [ ]:
_, axes = plt.subplots(d, d, figsize=(8, 8), dpi = 100,
                       subplot_kw={'xticks': [], 'yticks': []},
                       gridspec_kw=dict(hspace=0.1, wspace=0.1))

for ax, y in zip(axes.flat, manifold2):
    ax.plot(vdc,y.squeeze())
    ax.set_ylim(0.5, 1)

## Train two VAEs with enforced similarity b/w latent spaces

Now, let's define an ```imspec``` class that learns a linear mapping (in a form of coefficients of a linear equation) between two latent spaces durign the training. The pass forward then consists of the following steps:
1. Pass $x_1$ through *encoder-1*. Get latent representation $z_1$. Compute KLD. Pass $z_1$ through *decoder-1*. Compute reconstruction loss.
2. Pass $x_2$ through *encoder-2*. Get latent representation $z_2$. Compute KLD (the latter is invariant under affine trasformation).
3. Get the $z_2' = Az_2$ transformation. Compute the $L_1(z_1, z_2')$ score.
4. Pass $z_2'$ through *decoder-2* and compute a reconstruction loss.

In [ ]:
#@title imspec3
class imspec3(imspec):

    def __init__(self,
                 indim1: Tuple[int, int], # assumed to be images
                 indim2: Tuple[int], # assumed to be spectra
                 latent_dim: int = 2,  # for now, both VAEs will have the same number of latent dims
                 coord: int = 0,
                 **kwargs: torch.Tensor) -> None:
        # Initilize encoders and decoders
        args = (indim1, indim2, latent_dim, coord)
        super(imspec3, self).__init__(*args, **kwargs)
        # Define the (learnable) linear transformation coefficients
        A = torch.zeros(latent_dim, latent_dim).fill_diagonal_(1)
        A = A.to(self.device)
        self.A = nn.Parameter(A)

    @classmethod
    def _compute_l1(cls,
                    x1: torch.Tensor,
                    x2: torch.Tensor) -> torch.Tensor:
        """Computes L1 score"""
        return (x1 -x2).abs().mean()

    def linear_transform(self, z: torch.Tensor):
        """Performs a linear transformation of the latent encoding"""
        return z @ self.A

    def forward(self, x1: torch.Tensor, x2: torch.Tensor) -> torch.Tensor:
        """Forward Pass"""

        self.likelihood = 0
        self.kl_divergence = 0

        # First VAE
        # Encode data
        z_mu1, z_logsig1 = self.encoder1(x1)
        # Obtain latent vector throught he 'reparametrization trick'
        z_vec1 = self._reparameterize(z_mu1, z_logsig1)
        # Split latent vector into parts associated with coordinate transformations and data content
        t_vec1 = z_vec1[:, :self.coord]
        z_vec1 = z_vec1[:, self.coord:]
        # Compute KLD for the 'conventional' latent vector
        self.kl_divergence += self._compute_kld(
            z_vec1, (z_mu1[:, self.coord:], z_logsig1[:, self.coord:]))
        # Compute KLD for the latent angle (if any)
        if self.coord in [1, 3]:
            p_loc = t_vec1.new_zeros(t_vec1.size(0), 1)
            p_scale = t_vec1.new_ones(t_vec1.size(0), 1) * self.phi_prior
            self.kl_divergence += self._compute_kld(
                t_vec1[:, :1], (z_mu1[:, :1], z_logsig1[:, :1]), (p_loc, p_scale))
            z_mu1, z_logsig1 = z_mu1[:, 1:], z_logsig1[:, 1:]
            t_vec1 = t_vec1[:, 1:] if t_vec1.size(-1) > 1 else t_vec1
        # Compute KLD for the latent shift (if any)
        if self.coord in [2, 3]:
            self.kl_divergence += self._compute_kld(
                t_vec1, (z_mu1[:, :2], z_logsig1[:, :2])) # assume unit normal prior
        # Do the affine transformation for the coordinate grid (if any)
        decoder_input = (z_vec1,)
        if self.coord:
            grid = self.grid.expand(t_vec1.shape[0], *self.grid.shape)
            grid = self._transform_coordinates(grid, t_vec1)
            decoder_input = (grid,) + decoder_input
        # Pass latent data and transformed grid (if any) to decoder
        x_reconstr1 = self.decoder1(*decoder_input)
        # Compute reconstruction loss
        self.likelihood += -self._compute_re(x1, x_reconstr1)

        # Second VAE
        z_mu2, z_logsig2 = self.encoder2(x2)
        z_mu2 = self.linear_transform(z_mu2) # Do linear transformation

        z_vec2 = self._reparameterize(z_mu2, z_logsig2)
        self.kl_divergence +=  self._compute_kld(z_vec2, (z_mu2, z_logsig2))

        self.simloss = self._compute_l1(z_vec1, z_vec2)  # Compute L1 score between two latent spaces

        x_reconstr2 = self.decoder2(z_vec2)
        self.likelihood += -self._compute_re(x2, x_reconstr2)

        return (z_vec1, x_reconstr1), (z_vec2, x_reconstr2)

Initialize, model, optimizer, and dataloader:

In [ ]:
img_dim = (imgdata.shape[-1], imgdata.shape[-1])
spec_dim = (specdata.shape[-1],)

# (re-)initialize dat loader
train_loader = init_dataloader(imgdata.unsqueeze(1), specdata.unsqueeze(1), batch_size=200)

# initialize model and optimizer
set_torch_seed(1)
model = imspec3(img_dim, spec_dim, latent_dim=2, coord=3)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4 )

Next we train our new model. Note that the main difference here is that we add a ```simloss``` term to the loss function. We also add a ```gamma``` coefficients before this new loss term which controls it's relative contribution to the total loss (similar to the idea of beta-VAE).

In [ ]:
def train_step(x1: torch.Tensor, x2: torch.Tensor,
               beta: float = 1., gamma: float = 10.) -> float:
    model.train()
    _ = model(x1.to(model.device), x2.to(model.device))
    loss = -(model.likelihood - beta * model.kl_divergence).mean() + gamma * model.simloss
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return loss.item(), model.simloss.item()

train_loss, sim = [], []
for e in range(epochs):
    train_loss_i, sim_i = 0, 0
    for (x1, x2) in train_loader:
        loss_i, sscore_i = train_step(x1, x2)
        train_loss_i += loss_i
        sim_i += sscore_i
    train_loss.append(train_loss_i / len(train_loader))
    sim.append(sim_i / len(train_loader))
    print("Epoch {},  Train loss {}".format(e+1, train_loss[-1]))

Encode data into the latent spaces. Note that we have an extra line that applies a learned transformation to the encoded $z$.

In [ ]:
test_loader = init_dataloader(imgdata.unsqueeze(1), specdata.unsqueeze(1), shuffle=False)

model.eval()
with torch.no_grad():
    z_mu1 = torch.cat([model.encoder1(x.to(model.device))[0].cpu() for (x,_) in test_loader])
    z_mu2 = torch.cat([model.encoder2(x.to(model.device))[0].cpu() for (_,x) in test_loader])
    z_mu2 = z_mu2.matmul(model.A.cpu())  # a.matmul(b) is equivalent to a @ b and to torch.matmul(a, b)

plt.figure (figsize = (5,5), dpi = 200)
plt.scatter(z_mu1[:, -1], z_mu1[:, -2], s=24, alpha=0.3, label="image data")
plt.scatter(z_mu2[:, -1], z_mu2[:, -2], s=24, alpha=0.3, label="spectral data")
plt.xlabel ('$Z$$_1$')
plt.ylabel ('$Z$$_2$')
plt.legend()

View the learned latent manifolds. Note that we have an extra line that applies a learned transformation to the $z$ vector.

In [ ]:
d = 10
z, (gridx, gridy) = generate_latent_grid(d)
z_t = torch.matmul(z, model.A.cpu())
coord_grid = model.grid.expand(len(z), *model.grid.shape)

with torch.no_grad():
    manifold1 = model.decoder1(coord_grid, z.to(model.device)).cpu()
    manifold2 = model.decoder2(z_t.to(model.device)).cpu()

In [ ]:
grid = torchvision.utils.make_grid(manifold1, nrow=d, padding=1, pad_value=0.8)[0]
plt.figure(figsize=(6,6), dpi=200)
plt.imshow(grid, cmap='gray')
plt.axis("off")

In [ ]:
_, axes = plt.subplots(d, d, figsize=(10, 10),dpi = 100,
                       subplot_kw={'xticks': [], 'yticks': []},
                       gridspec_kw=dict(hspace=0.1, wspace=0.1))

for ax, y in zip(axes.flat, manifold2):
    ax.plot(vdc, y.squeeze(), c = 'blue')
    ax.set_ylim(0.1, 1.2)

Plot some example image-spectroscopy pairs

In [ ]:
plt.figure (figsize = (2,2), dpi =200)
plt.imshow(grid [59:59+28, 1:29], cmap = 'gray')
plt.xticks([])
plt.yticks([])

k = 30
plt.figure(figsize = (2, 2), dpi = 300)
plt.plot(vdc, manifold2[k,0,]-manifold2[k,0,].max(), c = "b")
plt.yticks([0.0, -0.5, -1.0])
plt.xticks([0.0, -1.0, -2.0])
plt.ylim(-1.1, 0.1)
plt.xlabel ("DC Voltage (V)")
plt.ylabel ("Normalized Current (a.u.)")

In [ ]:
plt.figure (figsize = (2,2), dpi =200)
plt.imshow(grid [30:30+28, 146:146+28], cmap = 'gray')
plt.xticks([])
plt.yticks([])

k = 15
plt.figure(figsize = (2, 2), dpi = 300)
plt.plot(vdc, manifold2[k,0,]-manifold2[k,0,].max(), c = "b")
plt.yticks([])
plt.xticks([0.0, -1.0, -2.0])
plt.ylim(-1.1, 0.1)
plt.xlabel ("DC Voltage (V)")
# plt.ylabel ("Normalized Current (a.u.)")

In [ ]:
plt.figure (figsize = (2,2), dpi =200)
plt.imshow(grid [175-29:175-29+28, 175+29:175+29+28], cmap = 'gray')
plt.xticks([])
plt.yticks([])

k = 67
plt.figure(figsize = (2, 2), dpi = 300)
plt.plot(vdc, manifold2[k,0,]-manifold2[k,0,].max(), c = "b")
plt.yticks([])
plt.xticks([0.0, -1.0, -2.0])
plt.ylim(-1.1, 0.1)
plt.xlabel ("DC Voltage (V)")
# plt.ylabel ("Normalized Current (a.u.)")

In [ ]:
plt.figure (figsize = (2,2), dpi =200)
plt.imshow(grid [-29:-1, 175-29:175-29+28], cmap = 'gray')
plt.xticks([])
plt.yticks([])

k = 95
plt.figure(figsize = (2, 2), dpi = 300)
plt.plot(vdc, manifold2[k,0,]-manifold2[k,0,].max(), c = "b")
plt.yticks([])
plt.xticks([0.0, -1.0, -2.0])
plt.ylim(-1.1, 0.1)
plt.xlabel ("DC Voltage (V)")

Let's also define a function that takes images (spectra) as an input at outputs spectra (images) and try it on the image -> spectrum predictions.

In [ ]:
#@title hybrid_fwd
def hybrid_fwd(model: Type[torch.nn.Module], x: torch.Tensor,
               out: str) -> torch.Tensor:
    """
    Passes the input features through the encoder of the first (second) model to obtain
    a latent code which is then passed through the decoder of the second (first) model
    """
    if out not in ['image', 'spectrum']:
        raise NotImplementedError("argument 'out' must be 'image' or 'spectrum'")
    x = x.to(model.device)
    model.eval()
    with torch.no_grad():
        z_mu = model.encoder2(x)[0] @ model.A if out == 'image' else model.encoder1(x)[0]
        z_mu = z_mu[:, model.coord:] if out == 'spectrum' else z_mu
        dec1_input = (z_mu,)
        if model.coord > 0:
            grid = model.grid.expand(z_mu.size(0), *model.grid.size())
            dec1_input = (grid,) + dec1_input
        x_rec = model.decoder1(*dec1_input) if out == 'image' else model.decoder2(z_mu)
    return x_rec.cpu()


In [ ]:
for i in range (20):
  k = np.random.choice(len(specdata))

  x_input = imgdata[k]
  x_input = x_input[None, None] # add two pseudo-dimensions for batch size and channel
  x_pred = hybrid_fwd(model, x_input, out='spectrum')

  _, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(14, 4), dpi = 100)
  ax1.imshow(x_input.squeeze())
  ax1.axis("off")
  ax1.set_title("Structure")
  ax2.plot(vdc, x_pred.squeeze(), label = "prediction")
  ax2.set_title("Spectrum--Prediction")
  ax2.set_xlabel("Voltage (V)")
  ax2.set_ylabel("Current (norm.)")
  #ax2.set_ylim(0.25, 0.35)

  ax3.set_title("Spectrum--Ground truth")
  ax3.set_xlabel("Voltage (V)")
  ax3.set_ylabel("Current (norm.)")
  #ax3.set_ylim(0.25, 0.35)
  ax3.plot(vdc, specdata[k].squeeze(), color = "darkorange")
  plt.show()

We can also try predicting image from spectra.

In [ ]:
for i in range (20):
  k = np.random.choice(len(specdata))
  x_input = specdata[k]
  x_input = x_input[None, None] # add two pseudo-dimensions for batch size and channel
  x_pred = hybrid_fwd(model, x_input, out='image')

  _, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(14, 4), dpi = 100)
  ax2.imshow(x_pred.squeeze())
  ax2.axis("off")
  ax2.set_title("Structure--Prediction")

  ax3.imshow(imgdata[k].squeeze())
  ax3.axis("off")
  ax3.set_title("Structure--Ground truth")

  ax1.plot(vdc,x_input.squeeze())
  ax1.set_xlabel("Voltage (V)")
  ax1.set_ylabel("Current (Normalized)")
  ax1.set_title("Spectrum")
  plt.show()